# Проект по SQL

Коронавирус застал мир врасплох, изменив привычный порядок вещей. На какое-то время жители городов перестали выходить на улицу, посещать кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.


Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных.


В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

Описание данных:
* Таблица books
* Содержит данные о книгах:
    * `book_id` — идентификатор книги;
    * `author_id` — идентификатор автора;
    * `title` — название книги;
    * `num_pages` — количество страниц;
    * `publication_date` — дата публикации книги;
    * `publisher_id` — идентификатор издателя.
* Таблица authors
* Содержит данные об авторах:
    * `author_id` — идентификатор автора;
    * `author` — имя автора.
* Таблица publishers
* Содержит данные об издательствах:
    * `publisher_id` — идентификатор издательства;
    * `publisher` — название издательства;
* Таблица ratings
* Содержит данные о пользовательских оценках книг:
    * `rating_id` — идентификатор оценки;
    * `book_id` — идентификатор книги;
    * `username` — имя пользователя, оставившего оценку;
    * `rating` — оценка книги.
* Таблица reviews
* Содержит данные о пользовательских обзорах:
    * `review_id` — идентификатор обзора;
    * `book_id` — идентификатор книги;
    * `username` — имя автора обзора;
    * `text` — текст обзора.

In [1]:
import pandas as pd 
from sqlalchemy import text, create_engine 
# устанавливаем параметры 
db_config = {'user': 'praktikum_student', # имя пользователя 
'pwd': 'Sdf4$2;d-d30pp', # пароль 
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net', 
'port': 6432, # порт подключения 
'db': 'data-analyst-final-project-db'} # название базы данных 
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config) 
 # сохраняем коннектор 
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 
con=engine.connect()

# чтобы выполнить SQL-запрос, используем Pandas 
query = '''SELECT * FROM books LIMIT 5''' 
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


# Исследование данных

In [2]:
query = ''' 
SELECT * FROM books
LIMIT 1
'''
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93


In [3]:
query = ''' 
SELECT * FROM authors
LIMIT 1
'''
pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt


In [4]:
query = ''' 
SELECT * FROM ratings
LIMIT 1
'''
pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4


In [5]:
query = ''' 
SELECT * FROM reviews
LIMIT 1
'''
pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...


In [6]:
query = ''' 
SELECT * FROM publishers
LIMIT 1
'''
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace


# Решение поставленных задач

## Посчитайте, сколько книг вышло после 1 января 2000 года

In [7]:
query = ''' 
SELECT COUNT(book_id) FROM books WHERE publication_date > '2000-01-01'
'''
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,819


После 1 января 2000 года было выпущено 821 книга

## Для каждой книги посчитайте количество обзоров и среднюю оценку

In [14]:
query = '''
SELECT 
    books.book_id, 
    books.title,
    cnt, 
    AVG(un.rt) AS avg_rating
FROM books
LEFT JOIN (SELECT 
           book_id, 
           COUNT(review_id) AS cnt
           FROM reviews 
           GROUP BY book_id) as otz ON otz.book_id = books.book_id
LEFT JOIN (SELECT 
           book_id, 
           AVG(rating) AS rt 
           FROM ratings 
           GROUP BY book_id) as un ON un.book_id = books.book_id
GROUP BY 
    books.book_id, 
    cnt
ORDER BY 
    avg_rating DESC, cnt DESC
'''
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,title,cnt,avg_rating
0,672,The Cat in the Hat and Other Dr. Seuss Favorites,NaN,5.00
1,17,A Dirty Job (Grim Reaper #1),4.0,5.00
2,553,School's Out—Forever (Maximum Ride #2),3.0,5.00
3,444,Moneyball: The Art of Winning an Unfair Game,3.0,5.00
4,699,The Demon-Haunted World: Science as a Candle i...,2.0,5.00
...,...,...,...,...
995,915,The World Is Flat: A Brief History of the Twen...,3.0,2.25
996,202,Drowning Ruth,3.0,2.00
997,316,His Excellency: George Washington,2.0,2.00
998,371,Junky,2.0,2.00


Почти у всех книг очень мало отзывов, это плодит множество книг с оценками 4.0 - 5.0. Также в базе данных содержатся книги с одинаковым названием, но разными авторами

## Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так исключим из анализа брошюры

In [9]:
query = ''' 
SELECT
    publisher,
    COUNT(*) AS count_books
FROM books
INNER JOIN publishers ON publishers.publisher_id = books.publisher_id
WHERE num_pages > 50
GROUP BY publisher
ORDER BY count_books DESC
LIMIT 1;
'''
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher,count_books
0,Penguin Books,42


Больше всего НЕ брошюр выпустило издательство Pinguin Books. 42 штуки

## Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;

In [10]:
query = ''' 
SELECT avgs.author,
       AVG(avgs.avg)
FROM (SELECT author, AVG(ratings.rating)
FROM books
INNER JOIN authors ON authors.author_id = books.author_id
INNER JOIN ratings ON ratings.book_id = books.book_id
GROUP BY books.book_id, authors.author_id
HAVING COUNT(rating_id) >= 50) AS avgs       
GROUP BY avgs.author
ORDER BY avg DESC
'''
pd.io.sql.read_sql(sql=text(query), con = con)

,author,avg
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645
5,William Golding,3.901408
6,J.D. Salinger,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,Dan Brown,3.754540


Самый высший средний рейтинг среди авторов, которые имеет более 50 оцено - J.K. Rowling/Mary GrandPré

## Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [11]:
query = ''' 

SELECT AVG(count)
FROM (SELECT username,
       COUNT(review_id)
FROM reviews
WHERE username in (SELECT username
FROM ratings
GROUP BY username
HAVING COUNT(rating_id) > 48)
GROUP BY username) AS many


'''
pd.io.sql.read_sql(sql=text(query), con = con)

,avg
0,24.0


В среднем пользователи, которые поставили более 48 оценок имеют ровно 24 отзыва

## Дополнительные запросы

In [12]:
query = ''' 
SELECT COUNT(book_id) FROM books WHERE publication_date < '2000-01-01'
'''
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,179


# Общий вывод

Согласно данным, полученным в исследовании, можно утверждать:

    1) После 01-01-2001 заметин резкий скачок в количестве выпущенных книг примерно в 4 раза. Возможно, держателям стартапа стоит ввести модерацию и не заливать с свой сервис каждую выпущенную книгу, дабы снизить нагрузку на хранилища
    2) У каждой книги очень мало отзывов, что приводит к заполнению базы книгами с чрезвычайно высокими оценками Возможно, стит поощрять пользователей за оставление оценки
    3) Больше всего НЕ брошюр выпустило издательство Penguin Books. Возможно, стоит начать сотрудничать с ним напрямую
    4) Самые высокооцененные авторы J.K. Rowling/Mary GrandPré, Markus Zusak/Cao Xuân Việt Khương, J.R.R. Tolkien. Для повышения популярности стартапа будет не лишним добавить в сервис больше книг за их авторством
    5) Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок - 24.0